In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Now let's try to expand it to the whole run
start = Time("2023-03-23T09:40:00", scale='utc')
end = Time("2023-03-23T10:00:00", scale='utc')

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    

az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)   
print(len(az_track), len(el_track))

In [ ]:
az['actualPosition'].plot()

In [ ]:
azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
azPos = azPos[azPos['inPosition']] # Select only the True values
elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
elPos = elPos[elPos['inPosition']] # Select only the True values
print(len(azPos), len(elPos))

In [ ]:
# Find all of the time stamps

# Start with start_slew times

azs = az_track.values[:,0]
els = el_track.values[:,0]
times = az_track.values[:,1]
start_slew_times_1 = []
slew_dist_1 = []
slew_times_1 = []
for i in range(1,len(az_track)):
    az_shift = abs(azs[i] - azs[i-1])
    el_shift = abs(els[i] - els[i-1])
    if (az_shift > 0.1) or (el_shift > 0.1):
        start_slew_times_1.append(times[i])
        az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
        shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
        slew_dist_1.append(shift)
print(len(start_slew_times_1))

# Now in position timestamps

inPos_1 = []
azPosValues = azPos.values[:,1]
elPosValues = elPos.values[:,1]

for i in range(len(azPos)):
    if azPosValues[i] > elPosValues[i]:
        inPos_1.append(azPosValues[i])
    else:
        inPos_1.append(elPosValues[i])

print(len(inPos_1))

# Now pair them up

pairMin = 1.0
pairMax = 6.0

start_slew_times = []
slew_dist = []
slew_times = []
inPos = []

for i in range(len(start_slew_times_1)):
    for j in range(len(inPos_1)):
        deltaT = inPos_1[j] - start_slew_times_1[i] 
        if deltaT > pairMin and deltaT < pairMax:
            inPos.append(inPos_1[j])
            start_slew_times.append(start_slew_times_1[i])
            slew_times.append(deltaT)
            slew_dist.append(slew_dist_1[i])
        
print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))

In [ ]:
# Plotting the mount plots
fig = plt.figure(figsize = (8,10))
plt.subplots_adjust(hspace=0.7)
for index in [5]:#range(len(inPos) - 1):

    plotAz = az[(az['timestamp'] > inPos[index]-10.0) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]-10.0) & (el['timestamp'] < start_slew_times[index+1])]
    ss_time = Time(start_slew_times[index+1], format='unix_tai', scale='utc').isot
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot

    fig.clear()
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
    plt.subplot(4,2,1)
    ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
    ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax1.set_title(f"Azimuth")
    ax1.set_ylabel("Degrees")
    ax1.legend()
    plt.subplot(4,2,2)
    ax2 = plotEl['actualPosition'].plot(legend=True, color='green')
    ax2.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax2.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax2.set_title(f"Elevation")
    ax2.set_ylabel("Degrees")
    ax2.legend()
    
    plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < start_slew_times[index+1])]

    plotAzTrack = az_track[(az_track['taiTime'] > inPos[index]) & (az_track['taiTime'] < start_slew_times[index+1])]
    plotElTrack = el_track[(el_track['taiTime'] > inPos[index]) & (el_track['taiTime'] < start_slew_times[index+1])]
    
    plt.subplot(4,2,3)
    plt.title("Azimuth")
    plt.plot(plotAz.values[:,1], plotAz.values[:,0], color='red', label='Measured')
    plt.plot(plotAzTrack.values[:,1], plotAzTrack.values[:,0], color='blue', ls='--', label='Commanded')
    plt.ylabel("Degrees")
    plt.legend()
    plt.subplot(4,2,4)
    plt.title("Elevation")
    plt.plot(plotEl.values[:,1], plotEl.values[:,0], color='green', label='Measured')
    plt.plot(plotElTrack.values[:,1], plotElTrack.values[:,0], color='magenta', ls='--', label='Commanded')
    plt.ylabel("Degrees")
    plt.legend()

    midTime = (inPos[index] + start_slew_times[index+1]) / 2.0
    plotAzBlowUp = az[(az['timestamp'] > midTime-0.2) & (az['timestamp'] < midTime+0.2)]
    plotElBlowUp = el[(el['timestamp'] > midTime-0.2) & (el['timestamp'] < midTime+0.2)]
    plotAzTrackBlowUp = az_track[(az_track['taiTime'] > midTime-0.2) & (az_track['taiTime'] < midTime+0.2)]
    plotElTrackBlowUp = el_track[(el_track['taiTime'] > midTime-0.2) & (el_track['taiTime'] < midTime+0.2)]

    azMin = np.min(plotAzBlowUp.values[:,0])
    azMax = np.max(plotAzBlowUp.values[:,0])
    elMin = np.min(plotElBlowUp.values[:,0])
    elMax = np.max(plotElBlowUp.values[:,0])
    plt.subplot(4,2,5)
    plt.title("Azimuth")
    plt.plot(plotAzBlowUp.values[:,1], plotAzBlowUp.values[:,0], color='red', label='Measured')
    plt.plot(plotAzTrackBlowUp.values[:,1], plotAzTrackBlowUp.values[:,0], color='blue', ls='--', label='Commanded')
    plt.ylabel("Degrees")
    plt.ylim(azMin, azMax)
    plt.legend()
    plt.subplot(4,2,6)
    plt.title("Elevation")
    plt.plot(plotElBlowUp.values[:,1], plotElBlowUp.values[:,0], color='green', label='Measured')
    plt.plot(plotElTrackBlowUp.values[:,1], plotElTrackBlowUp.values[:,0], color='magenta', ls='--', label='Commanded')
    plt.ylabel("Degrees")
    plt.ylim(elMin, elMax)
    plt.legend()
    
    # Calculate the difference between measured and commanded
    plotAzTrackAtPlotAzTimes = np.interp(plotAz.values[:,1], plotAzTrack.values[:,1], plotAzTrack.values[:,0])
    azErrors = (plotAz.values[:,0] - plotAzTrackAtPlotAzTimes) * 3600.0
    plotElTrackAtPlotElTimes = np.interp(plotEl.values[:,1], plotElTrack.values[:,1], plotElTrack.values[:,0])
    elErrors = (plotEl.values[:,0] - plotElTrackAtPlotElTimes) * 3600.0
    
    plt.subplot(4,2,7)
    plt.title("Measured - Commanded")
    plt.plot(plotAz.values[:,1], azErrors, color='r')
    plt.ylabel("Error(arcseconds)")
    plt.ylim(-1.0, 1.0)
    plt.subplot(4,2,8)
    plt.title("Measured - Commanded")
    plt.plot(plotEl.values[:,1], elErrors, color='g')
    plt.ylabel("Error(arcseconds)")
    plt.ylim(-1.0, 1.0)

    
    
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    plt.savefig(f"/home/craiglagegit/DATA/MTMount_22mar23/MT_Mount_Measured_vs_Commanded_BlowUp_{timestamp}.pdf")



In [ ]:
# Now let's try to expand it to the whole run
start = Time("2023-03-23T09:40:00", scale='utc')
end = Time("2023-03-23T10:00:00", scale='utc')

az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'demandPosition', 'timestamp'],  start, end)    

az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)   
print(len(az_track), len(el_track))

In [ ]:
az['actualPosition'].plot()

In [ ]:
azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
azPos = azPos[azPos['inPosition']] # Select only the True values
elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
elPos = elPos[elPos['inPosition']] # Select only the True values
print(len(azPos), len(elPos))

In [ ]:
# Find all of the time stamps

# Start with start_slew times

azs = az_track.values[:,0]
els = el_track.values[:,0]
times = az_track.values[:,1]
start_slew_times_1 = []
slew_dist_1 = []
slew_times_1 = []
for i in range(1,len(az_track)):
    az_shift = abs(azs[i] - azs[i-1])
    el_shift = abs(els[i] - els[i-1])
    if (az_shift > 0.1) or (el_shift > 0.1):
        start_slew_times_1.append(times[i])
        az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
        shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
        slew_dist_1.append(shift)
print(len(start_slew_times_1))

# Now in position timestamps

inPos_1 = []
azPosValues = azPos.values[:,1]
elPosValues = elPos.values[:,1]

for i in range(len(azPos)):
    if azPosValues[i] > elPosValues[i]:
        inPos_1.append(azPosValues[i])
    else:
        inPos_1.append(elPosValues[i])

print(len(inPos_1))

# Now pair them up

pairMin = 1.0
pairMax = 6.0

start_slew_times = []
slew_dist = []
slew_times = []
inPos = []

for i in range(len(start_slew_times_1)):
    for j in range(len(inPos_1)):
        deltaT = inPos_1[j] - start_slew_times_1[i] 
        if deltaT > pairMin and deltaT < pairMax:
            inPos.append(inPos_1[j])
            start_slew_times.append(start_slew_times_1[i])
            slew_times.append(deltaT)
            slew_dist.append(slew_dist_1[i])
        
print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))

In [ ]:
# Plotting the mount plots
fig = plt.figure(figsize = (8,10))
plt.subplots_adjust(hspace=0.7)
for index in [5]:#range(len(inPos) - 1):

    plotAz = az[(az['timestamp'] > inPos[index]-10.0) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]-10.0) & (el['timestamp'] < start_slew_times[index+1])]
    ss_time = Time(start_slew_times[index+1], format='unix_tai', scale='utc').isot
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot

    fig.clear()
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
    plt.subplot(4,2,1)
    ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
    ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax1.set_title(f"Azimuth")
    ax1.set_ylabel("Degrees")
    ax1.legend()
    plt.subplot(4,2,2)
    ax2 = plotEl['actualPosition'].plot(legend=True, color='green')
    ax2.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax2.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax2.set_title(f"Elevation")
    ax2.set_ylabel("Degrees")
    ax2.legend()
    
    plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < start_slew_times[index+1])]

    plt.subplot(4,2,3)
    plt.title("Azimuth")
    plt.plot(plotAz.values[:,2], plotAz.values[:,0], color='red', label='Measured')
    plt.plot(plotAz.values[:,2], plotAz.values[:,1], color='blue', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()
    plt.subplot(4,2,4)
    plt.title("Elevation")
    plt.plot(plotEl.values[:,2], plotEl.values[:,0], color='green', label='Measured')
    plt.plot(plotEl.values[:,2], plotEl.values[:,1], color='magenta', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()
    
    midTime = (inPos[index] + start_slew_times[index+1]) / 2.0
    plotAzBlowUp = az[(az['timestamp'] > midTime+0.3) & (az['timestamp'] < midTime+1.1)]
    plotElBlowUp = el[(el['timestamp'] > midTime-0.4) & (el['timestamp'] < midTime+0.4)]

    plt.subplot(4,2,5)
    plt.title("Azimuth")
    plt.plot(plotAzBlowUp.values[:,2], plotAzBlowUp.values[:,0], color='red', label='Measured')
    plt.plot(plotAzBlowUp.values[:,2], plotAzBlowUp.values[:,1], color='blue', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()
    plt.subplot(4,2,6)
    plt.title("Elevation")
    plt.plot(plotElBlowUp.values[:,2], plotElBlowUp.values[:,0], color='green', label='Measured')
    plt.plot(plotElBlowUp.values[:,2], plotElBlowUp.values[:,1], color='magenta', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()

    # Calculate the difference between measured and commanded
    azErrors = (plotAz.values[:,0] - plotAz.values[:,1]) * 3600.0
    elErrors = (plotEl.values[:,0] - plotEl.values[:,1]) * 3600.0
    
    plt.subplot(4,2,7)
    plt.title("Measured - Demand")
    plt.plot(plotAz.values[:,2], azErrors, color='r')
    plt.ylabel("Error(arcseconds)")
    plt.ylim(-1.0, 1.0)
    plt.subplot(4,2,8)
    plt.title("Measured - Demand")
    plt.plot(plotEl.values[:,2], elErrors, color='g')
    plt.ylabel("Error(arcseconds)")
    plt.ylim(-1.0, 1.0)

    
    
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    plt.savefig(f"/home/craiglagegit/DATA/MTMount_22mar23/MT_Mount_Measured_vs_Demand_{timestamp}.png")



In [ ]:
plotElBlowUp

In [ ]:
plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < start_slew_times[index+1])]
measured = plotEl.values[:,0]
demand = plotEl.values[:,1]

for i in range(len(measured)):
    delta = abs(measured[i] - demand[i])
    if delta > 1.0E-5:
        print(i, demand[i], delta)

In [ ]:
plotEl.iloc[369:372]

In [ ]:
def cleanDemand(df):
    measured = df.values[:,0]
    demand = df.values[:,1]
    new_df = df.copy(deep=True)
    for i in range(len(measured)):
        delta = abs(measured[i] - demand[i])
        if delta > 1.0E-5:
            new_df.iloc[i, df.columns.get_loc('demandPosition')] = (demand[i-1] + demand[i+1]) / 2.0
            new_df.iloc[i, df.columns.get_loc('actualPosition')] = (measured[i-1] + measured[i+1]) / 2.0
    return new_df


In [ ]:
plotEl.columns

In [ ]:
plotEl = cleanDemand(plotEl)

In [ ]:
plotEl.iloc[369:372]

In [ ]:
# Plotting the mount plots
fig = plt.figure(figsize = (8,10))
plt.subplots_adjust(hspace=0.7)
for index in [5]:#range(len(inPos) - 1):

    plotAz = az[(az['timestamp'] > inPos[index]-10.0) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]-10.0) & (el['timestamp'] < start_slew_times[index+1])]
    ss_time = Time(start_slew_times[index+1], format='unix_tai', scale='utc').isot
    ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot

    fig.clear()
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
    plt.subplot(4,2,1)
    ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
    ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax1.set_title(f"Azimuth")
    ax1.set_ylabel("Degrees")
    ax1.legend()
    plt.subplot(4,2,2)
    ax2 = plotEl['actualPosition'].plot(legend=True, color='green')
    ax2.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax2.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    ax2.set_title(f"Elevation")
    ax2.set_ylabel("Degrees")
    ax2.legend()
    
    plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < start_slew_times[index+1])]
    plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < start_slew_times[index+1])]
    plotAz = cleanDemand(plotAz)
    plotEl = cleanDemand(plotEl)
    plt.subplot(4,2,3)
    plt.title("Azimuth")
    plt.plot(plotAz.values[:,2], plotAz.values[:,0], color='red', label='Measured')
    plt.plot(plotAz.values[:,2], plotAz.values[:,1], color='blue', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()
    plt.subplot(4,2,4)
    plt.title("Elevation")
    plt.plot(plotEl.values[:,2], plotEl.values[:,0], color='green', label='Measured')
    plt.plot(plotEl.values[:,2], plotEl.values[:,1], color='magenta', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()
    
    midTime = (inPos[index] + start_slew_times[index+1]) / 2.0
    plotAzBlowUp = az[(az['timestamp'] > midTime+0.3) & (az['timestamp'] < midTime+1.1)]
    plotElBlowUp = el[(el['timestamp'] > midTime-0.4) & (el['timestamp'] < midTime+0.4)]
    plotAzBlowUp = cleanDemand(plotAzBlowUp)
    plotElBlowUp = cleanDemand(plotElBlowUp)

    plt.subplot(4,2,5)
    plt.title("Azimuth")
    plt.plot(plotAzBlowUp.values[:,2], plotAzBlowUp.values[:,0], color='red', label='Measured')
    plt.plot(plotAzBlowUp.values[:,2], plotAzBlowUp.values[:,1], color='blue', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()
    plt.subplot(4,2,6)
    plt.title("Elevation")
    plt.plot(plotElBlowUp.values[:,2], plotElBlowUp.values[:,0], color='green', label='Measured')
    plt.plot(plotElBlowUp.values[:,2], plotElBlowUp.values[:,1], color='magenta', ls='--', label='Demand')
    plt.ylabel("Degrees")
    plt.legend()

    # Calculate the difference between measured and commanded
    azErrors = (plotAz.values[:,0] - plotAz.values[:,1]) * 3600.0
    elErrors = (plotEl.values[:,0] - plotEl.values[:,1]) * 3600.0
    
    plt.subplot(4,2,7)
    plt.title("Measured - Demand")
    plt.plot(plotAz.values[:,2], azErrors, color='r')
    plt.ylabel("Error(arcseconds)")
    plt.ylim(-0.05,0.05)
    plt.subplot(4,2,8)
    plt.title("Measured - Demand")
    plt.plot(plotEl.values[:,2], elErrors, color='g')
    plt.ylabel("Error(arcseconds)")
    plt.ylim(-0.05,0.05)

    
    
    timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
    #plt.savefig(f"/home/craiglagegit/DATA/MTMount_22mar23/MT_Mount_Measured_vs_Demand_{timestamp}.png")

